In [0]:
import pyspark
#from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
#from pyspark.sql import SparkSession, functions as F
#from pyspark.sql.functions import regexp_replace
#from pyspark import SparkContext, SparkConf

In [0]:
#Inicialização de variáveis
spark = SparkSession.builder.appName("ERPSpark").getOrCreate()

dict_table_key = {'customers': ['customer_number'], 
                  'employees': ['employee_number'], 
                  'offices': ['office_code'], 
                  'orderdetails': ['order_line_number', 'order_number'], 
                  'orders': ['order_number'], 
                  'payments': ['check_number'], 
                  'product_lines': ['product_line'], 
                  'products': ['product_code']}

driver = "org.postgresql.Driver"

database_host = "psql-mock-database-cloud.postgres.database.azure.com"
database_port = "5432"
database_name = "ecom1692290925429vkltgmdlpclbtrmh"
table = "table-name"
user = "mttdnzukfyledzwzollrcydx@psql-mock-database-cloud"
password = "anljjxxmzjlpbtvkvegkwnyt"

url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"                  

In [0]:
# Salvar cada tabela como parquet
for table in dict_table_key.keys():
    df = (spark.read
            .format("jdbc")
            .option("driver", driver)
            .option("url", url)
            .option("dbtable", table)
            .option("user", user)
            .option("password", password)
            .load()
         )
    
    df.write.mode("overwrite").parquet("/parquet/" + table + ".parquet")

#parDF1=spark.read.parquet("/parquet/product_lines.parquet")
#parDF1.show()

In [0]:
#merge entre a tabela JDBC e os arquivos parquet, o merge deve conter a lógica de insert, update e delete.
for table, p_key in dict_table_key.items():
    (spark.read
         .format("jdbc")
         .option("driver", driver)
         .option("url", url)
         .option("dbtable", table)
         .option("user", user)
         .option("password", password)
         .load()
         .createOrReplaceTempView(table + "_jdbc"))
    
    spark.read.parquet("/parquet/" + table + ".parquet").write.format("delta").mode("overwrite").save("/tmp/delta/" + table)
    spark.read.load("/tmp/delta/" + table).createOrReplaceTempView(table + "_delta")

    lst_columns = spark.sql("select * from " + table + "_delta limit 1").columns
    str_set = ', '.join('TARGET.{0}=SOURCE.{0}'.format(c) for c in lst_columns)
    str_columns = ', '.join(lst_columns)
    str_key = ' AND '.join('TARGET.{0}=SOURCE.{0}'.format(c) for c in p_key)

    # str_set = ''
    # str_columns = ''
    # for c in lst_columns:
    #     str_set += 'TARGET.{0}=SOURCE.{0}, '.format(c)
    #     str_columns += c + ', '
    # str_set = str_set[:-2]
    # str_columns = str_columns[:-2]

    

    sql = """
        MERGE INTO {0} AS TARGET
        USING {1} AS SOURCE
        ON {2}
        WHEN MATCHED AND 1=1 THEN 
        UPDATE SET {3}
        WHEN NOT MATCHED BY TARGET THEN
        INSERT ({4})
        VALUES ({4})
        WHEN NOT MATCHED BY SOURCE THEN
        DELETE
        """.format(table + "_delta", table + "_jdbc", str_key, str_set, str_columns)
    print(table)
    spark.sql(sql)
    

    spark.sql("select * from " + table + "_delta").write.mode("overwrite").parquet("/parquet/" + table + ".parquet")


customers
employees
offices
orderdetails
orders
payments
product_lines
products


In [0]:
# df = (spark.read
#             .format("jdbc")
#             .option("driver", driver)
#             .option("url", url)
#             .option("dbtable", "customers")
#             .option("user", user)
#             .option("password", password)
#             .load()
#          )

#df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/Data/df.csv")
#df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df.csv")
#/dbfs/FileStore/NJ/file1.txt
#df.write.format("delta").mode("overwrite").save("/delta/diamonds")

In [0]:
#- Qual país possui a maior quantidade de itens cancelados?
spark.read.parquet("/parquet/orders.parquet").createOrReplaceTempView("orders")
spark.read.parquet("/parquet/customers.parquet").createOrReplaceTempView("customers")
spark.read.parquet("/parquet/orderdetails.parquet").createOrReplaceTempView("orderdetails")

sql = """
    select sum(ord_det.quantity_ordered) qtde_cancelados, country 
      from orders ord
inner join customers cus on cus.customer_number = ord.customer_number
inner join orderdetails ord_det on ord_det.order_number = ord.order_number
     where ord.status = 'Cancelled'
  group by country
  order by 1 desc
     limit 1
"""     

spark.sql(sql).show()

+---------------+-------+
|qtde_cancelados|country|
+---------------+-------+
|            605|  Spain|
+---------------+-------+



In [0]:
#- Qual país possui a maior quantidade de itens cancelados?
spark.read.parquet("/parquet/orders.parquet").createOrReplaceTempView("orders")
spark.read.parquet("/parquet/customers.parquet").createOrReplaceTempView("customers")
spark.read.parquet("/parquet/orderdetails.parquet").createOrReplaceTempView("orderdetails")

sql = """
    select sum(ord_det.quantity_ordered) qtde_cancelados, country 
      from orders ord
inner join customers cus on cus.customer_number = ord.customer_number
inner join orderdetails ord_det on ord_det.order_number = ord.order_number
     where ord.status = 'Cancelled'
  group by country
  order by 1 desc
     limit 1
"""     

#spark.sql(sql).show()
spark.sql(sql).write.format("delta").mode("overwrite").save("/delta/result_pais_cancelados")

In [0]:
#- Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005?
spark.read.parquet("/parquet/product_lines.parquet").createOrReplaceTempView("product_lines")
spark.read.parquet("/parquet/products.parquet").createOrReplaceTempView("products")
spark.read.parquet("/parquet/orders.parquet").createOrReplaceTempView("orders")
spark.read.parquet("/parquet/orderdetails.parquet").createOrReplaceTempView("orderdetails")

sql = """
    select pl.product_line, sum(od.quantity_ordered * od.price_each) as faturamento 
      from product_lines pl
inner join products p on p.product_line = pl.product_line
inner join orderdetails od on od.product_code = p.product_code
inner join orders o on o.order_number = od.order_number
     where o.status = 'Shipped' and extract(year from o.order_date) = 2005
group by pl.product_line
order by 2 desc
   limit 1
"""     

#spark.sql(sql).show()
spark.sql(sql).write.format("delta").mode("overwrite").save("/delta/result_faturamento_produto")

In [0]:
#- Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado.
spark.read.parquet("/parquet/employees.parquet").createOrReplaceTempView("employees")
spark.read.parquet("/parquet/offices.parquet").createOrReplaceTempView("offices")

sql = """
    select first_name, last_name, replace(email, split_part(email, '@', 1), '########') email
      from employees emp
inner join offices off on off.office_code = emp.office_code
where off.country = 'Japan'
"""     

#spark.sql(sql).show()
spark.sql(sql).write.format("delta").mode("overwrite").save("/delta/result_vend_japao")

In [0]:
spark.read.parquet("/parquet/products.parquet").createOrReplaceTempView("products_parquet")

spark.sql('select * from products_parquet order by 1 desc').show()

+------------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+---------+------+
|product_code|        product_name|    product_line|product_scale|      product_vendor| product_description|quantity_in_stock|buy_price|  msrp|
+------------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+---------+------+
|    S72_3212|          Pont Yacht|           Ships|         1:72|Unimax Art Galleries|Measures 38 inche...|              414|    33.30| 54.60|
|    S72_1253|    Boeing X-32A JSF|          Planes|         1:72|Motor City Art Cl...|10\" Wingspan wit...|             4857|    32.77| 49.66|
|   S700_4002|American Airlines...|          Planes|        1:700| Second Gear Diecast|Polished finish. ...|             8820|    36.27| 74.03|
|   S700_3962|      The Queen Mary|           Ships|        1:700|Welly Diecast Pro...|Exact replica. Wo...|             5088|    53.63|

In [0]:
df = spark.read.load("/tmp/delta/products")
display(df)

product_code,product_name,product_line,product_scale,product_vendor,product_description,quantity_in_stock,buy_price,msrp
S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front suspension, gear-shift lever, footbrake lever, drive chain, wheels and steering. All parts are particularly delicate due to their precise scale and require special care and attention.",7933,48.81,95.70
S10_1949,Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,7305,98.58,214.30
S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish.",6625,68.99,118.94
S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos and insignias, detachable rear wheelie bar, heavy diecast metal with resin parts, authentic multi-color tampo-printed graphics, separate engine drive belts, free-turning front fork, rotating tires and rear racing slick, certificate of authenticity, detailed engine, display stand\r\n, precision diecast replica, baked enamel finish, 1:10 scale model, removable fender, seat and tank cover piece for displaying the superior detail of the v-twin engine",5582,91.02,193.66
S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,3252,85.68,136.00
S10_4962,1962 LanciaA Delta 16V,Classic Cars,1:10,Second Gear Diecast,Features include: Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,6791,103.42,147.74
S12_1099,1968 Fods Mustang,Classic Cars,1:12,Autoart Studio Design,"Hood, doors and trunk all open to reveal highly detailed interior features. Steering wheel actually turns the front wheels. Color dark green.",68,95.34,194.57
S12_1108,2001 Ferrari Enzo,Classic Cars,1:12,Second Gear Diecast,Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,3619,95.59,207.80
S12_1666,1958 Setra Bus,Trucks and Buses,1:12,Welly Diecast Productions,"Model features 30 windows, skylights & glare resistant glass, working steering system, original logos",1579,77.90,136.67
S12_2823,2002 Suzuki XREO,Motorcycles,1:12,Unimax Art Galleries,"Official logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish.",9997,66.27,150.62
